https://dspy.ai/tutorials/games/
> pip install -U alfworld multiprocess --use-deprecated=legacy-resolver --no-build-isolation (does not download)
> alfworld-download

https://dspy.ai/tutorials/classification_finetuning/
xxpip install "sglang[all]"; pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/
> pip install sglang
> pip install -U torch transformers accelerate trl peft
https://huggingface.co/datasets/PolyAI/banking77

In [1]:
import dspy
import random
from dspy.datasets import DataLoader
from datasets import load_dataset

# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
]

random.Random(0).shuffle(raw_data)

d:\code\dspy\.venv\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
d:\code\dspy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import json

with open('data/raw_data.json', 'w') as f:
    json.dump(raw_data, f, indent=2)


TypeError: Object of type Example is not JSON serializable

In [5]:
import pandas as pd

# Ensure raw_data is properly structured with actual text and label values
df = pd.DataFrame(raw_data)

# Add header names explicitly
df.columns = ['text', 'label']

# Save with proper formatting
df.to_csv('data/raw_data.csv', index=False, encoding='utf-8', quoting=1)



In [2]:
len(CLASSES), CLASSES[:10]

(77,
 ['activate_my_card',
  'age_limit',
  'apple_pay_or_google_pay',
  'atm_support',
  'automatic_top_up',
  'balance_not_updated_after_bank_transfer',
  'balance_not_updated_after_cheque_or_cash_deposit',
  'beneficiary_not_allowed',
  'cancel_transfer',
  'card_about_to_expire'])

In [ ]:
unlabeled_trainset = [dspy.Example(text=x.text).with_inputs("text") for x in raw_data[:500]]

print(unlabeled_trainset[0])
print(unlabeled_trainset[1])
print(unlabeled_trainset[2])

Example({'text': 'If I find a card I lost do I need to dispose of it? Or can I re-active the card and continue to use it?'}) (input_keys={'text'})

In [4]:
from typing import Literal

classify = dspy.ChainOfThought(f"text -> label: Literal{CLASSES}")

In [5]:
# from dspy.clients.lm_local import LocalProvider

# student_lm_name = "meta-llama/Llama-3.2-1B-Instruct"
# student_lm = dspy.LM(model=f"openai/local:{student_lm_name}", provider=LocalProvider(), max_tokens=2000)
# teacher_lm = dspy.LM('openai/gpt-4o-mini', max_tokens=3000)

import dspy
student_lm = dspy.LM('ollama_chat/llama3.2:1b-instruct-q2_K', api_base='http://localhost:11434', api_key='')
teacher_lm = dspy.LM('ollama_chat/deepseek-r1:latest', api_base='http://localhost:11434', api_key='')

In [6]:
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

teacher_classify = classify.deepcopy()
teacher_classify.set_lm(teacher_lm)

In [ ]:
optimizer = dspy.BootstrapFinetune(num_threads=16)  # if you *do* have labels, pass metric=your_metric here!
classify_ft = optimizer.compile(student_classify, teacher=teacher_classify, trainset=unlabeled_trainset)

[BootstrapFinetune] Preparing the student and teacher programs...
[BootstrapFinetune] Bootstrapping data...
  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# classify_ft.get_lm().launch()
classify_ft(text="I didn't receive my money earlier and it says the transaction is still in progress. Can you fix it?")

In [ ]:
devset = raw_data[500:600]
devset[0]

In [ ]:
metric = (lambda x, y, trace=None: x.label == y.label)
evaluate = dspy.Evaluate(devset=devset, metric=metric, display_progress=True, display_table=5, num_threads=16)

In [ ]:
evaluate(student_lm)